# Import

In [1]:
import json
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta

# function to download stock data

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Write heading of the columns
        f.write('Date,Open,High,Low,Close,Volume\n')
        # load share data for the time period needed
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Choose the right columns
        stock_data.to_csv(f, header=False)  # Write the data to the file

# Define the shares

In [3]:
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']

# Read start and end for the sequence, which will be predicted from Excel

In [4]:
excel_file_path = 'settings/actualMonth_startEnd.xlsx'

# Path to file, in which the index is stored
index_file_path = 'settings/actualMonthIndex.txt'

# Read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0
print(current_row_index)
# read excel file
df = pd.read_excel(excel_file_path)

# Check, if the index is out of bounds
if current_row_index >= len(df):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract the data from the current row
    start_date = df.loc[current_row_index, 'start_date']
    end_date = df.loc[current_row_index, 'end_date']

    # Output of the current data
    print(f'Startdatum: {start_date}, Enddatum: {end_date}')

    # Updating the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

15
Startdatum: 2024-04-01, Enddatum: 2024-04-30


# Read close values at the begin of the time period

In [5]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = start_date_obj - timedelta(days=1)

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2024-03-27 00:00:00
2024-03-31 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)
    
# Add a row for the risk-free interest rate with the specified value
risk_free_rate = 4 # Input in percentage
risk_free_rate = risk_free_rate / 12 # Risk free rate per month
risk_free_data = {'Symbol': 'Risk_Free', 'Close_Value': 100}
combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)

print(combined_data_begin)

C:\Users\Samuel\AppData\Local\Temp\ipykernel_17300\18960333.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)


       Symbol  Close_Value
0      ALV.DE   277.799988
1      DBK.DE    14.582000
2     VOW3.DE   122.839996
3      BMW.DE   106.959999
4      ADS.DE   207.000000
5      BEI.DE   134.949997
6      DTE.SG    22.434999
7      SAP.DE   180.460007
8     1COV.DE    50.680000
9      BAS.DE    52.930000
10    EOAN.DE    12.885000
11     RWE.DE    31.459999
12  Risk_Free   100.000000


# read predictions

In [7]:
# Define the name of the file
json_file = 'settings/predictions_dict.json'

# read array from the json file
with open(json_file, 'r') as f:
    predictions_dict = json.load(f)
    
# Convert predictions_dict to DataFrame
predictions_df = pd.DataFrame(predictions_dict)
print(predictions_df)

                              ALV.DE     DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead  145.755386  14.388221  85.346313  87.201279   

                              ADS.DE      BEI.DE     DTE.SG     SAP.DE  \
prediction_20_days_ahead  108.782417  100.752472  20.497587  84.601852   

                            1COV.DE     BAS.DE    EOAN.DE     RWE.DE  
prediction_20_days_ahead  49.830753  50.684361  13.794801  31.657511  


# add dividend

In [8]:
excel_file_path_div = 'settings/Dividend2023.xlsx'
div = pd.read_excel(excel_file_path_div)
print(div)
# Divide all values by 12 in the column "Dividend"
div['Dividend'] = div['Dividend'] / 12

print(div)

# set symbol column as index
div.set_index('Share', inplace=True)

# Add the dividend for every symbol in preditions_df
for symbol in predictions_df.columns:
    if symbol in div.index:
        predictions_df[symbol] += div.loc[symbol, 'Dividend']

print(predictions_df)

      Share  Dividend
0    ALV.DE     11.40
1    DBK.DE      0.30
2   VOW3.DE     27.82
3    BMW.DE      8.50
4    ADS.DE      0.70
5    BEI.DE      0.70
6    DTE.SG      0.70
7    SAP.DE      2.05
8   1COV.DE      0.00
9    BAS.DE      3.40
10  EOAN.DE      0.51
11   RWE.DE      0.90
      Share  Dividend
0    ALV.DE  0.950000
1    DBK.DE  0.025000
2   VOW3.DE  2.318333
3    BMW.DE  0.708333
4    ADS.DE  0.058333
5    BEI.DE  0.058333
6    DTE.SG  0.058333
7    SAP.DE  0.170833
8   1COV.DE  0.000000
9    BAS.DE  0.283333
10  EOAN.DE  0.042500
11   RWE.DE  0.075000
                              ALV.DE     DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead  146.705386  14.413221  87.664647  87.909612   

                              ADS.DE      BEI.DE     DTE.SG     SAP.DE  \
prediction_20_days_ahead  108.840751  100.810805  20.555921  84.772686   

                            1COV.DE     BAS.DE    EOAN.DE     RWE.DE  
prediction_20_days_ahead  49.830753  50.967694  13.837301  31

# calculate return in %

In [9]:
# Für jedes Symbol in predictions_df den entsprechenden Close_Value aus combined_data_begin abziehen
for symbol in predictions_df.columns:
    if symbol in combined_data_begin['Symbol'].values:
        close_value = combined_data_begin.loc[combined_data_begin['Symbol'] == symbol, 'Close_Value'].values[0]
        predictions_df[symbol] = ((predictions_df[symbol] - close_value) / close_value) * 100

print(predictions_df)   

                             ALV.DE    DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead -47.190283 -1.157447 -28.635095 -17.810758   

                             ADS.DE    BEI.DE    DTE.SG     SAP.DE   1COV.DE  \
prediction_20_days_ahead -47.419927 -25.29766 -8.375658 -53.024115 -1.675704   

                            BAS.DE   EOAN.DE    RWE.DE  
prediction_20_days_ahead -3.707362  7.390768  0.866216  


# add risk free rate per month

In [10]:
# Symbol für den risikofreien Zinssatz
risk_free_symbol = 'Risk Free'

# Erstellen einer neuen Zeile für den risikofreien Zinssatz
risk_free_row = pd.DataFrame({risk_free_symbol: [(risk_free_rate)]}, index=['prediction_20_days_ahead'])

# Anhängen der neuen Zeile an predictions_df
predictions_df = predictions_df.join(risk_free_row)

print(predictions_df)


                             ALV.DE    DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead -47.190283 -1.157447 -28.635095 -17.810758   

                             ADS.DE    BEI.DE    DTE.SG     SAP.DE   1COV.DE  \
prediction_20_days_ahead -47.419927 -25.29766 -8.375658 -53.024115 -1.675704   

                            BAS.DE   EOAN.DE    RWE.DE  Risk Free  
prediction_20_days_ahead -3.707362  7.390768  0.866216   0.333333  


# read profit

In [11]:
json_file_path = 'settings/profit.json'

try:
    # read JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
        profit = data['profit']
except FileNotFoundError:
    # If the file can not be found, set the value of new_portfolio_value to 0
    profit = 0

print(f'Profit: {profit}')

Profit: 119.83800768359538


# load share value from 2019

In [12]:
start_date = start_date
end_date = end_date
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


Startdatum: 2024-04-01, Enddatum: 2024-04-30


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
# list for the last close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
    
# Add a row for the risk-free interest rate with a value of 1
last_close_values.append(('Risk_Free', 100 + risk_free_rate))
    
# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

Liste der letzten Close-Werte:
ALV.DE: 266.29998779296875
DBK.DE: 15.104000091552734
VOW3.DE: 120.75
BMW.DE: 106.8000030517578
ADS.DE: 232.3000030517578
BEI.DE: 139.4499969482422
DTE.SG: 21.71999931335449
SAP.DE: 171.4199981689453
1COV.DE: 47.54999923706055
BAS.DE: 49.05500030517578
EOAN.DE: 12.46500015258789
RWE.DE: 33.0
Risk_Free: 100.33333333333333


# Historic data from 2018 for the covarianz matrix

In [14]:
excel_file_path = 'settings/lastSequenceForPrediction.xlsx'

# path to the data, in which the index is stored
index_file_path = 'settings/lastSequenceForPrediction.txt'

# read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0

# read excel file
df_lastSeq = pd.read_excel(excel_file_path)

# Check, if index is out of bounds
if current_row_index >= len(df_lastSeq):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract data from the current row
    start_date_lastSeq = df_lastSeq.loc[current_row_index, 'start_date']
    end_date_lastSeq = df_lastSeq.loc[current_row_index, 'end_date']
    
    # Output of the current data 
    print(f'Startdatum: {start_date_lastSeq}, Enddatum: {end_date_lastSeq}')

    # Update the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

Startdatum: 2023-10-01, Enddatum: 2024-03-31


In [15]:
# Definition of the time period
start_date2018 = '2023-01-01'
end_date2018 = end_date_lastSeq
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2024-03-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file 

In [16]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)

# Initialize the initial and final values for "Risk_Free"
risk_free_initial = 100
risk_free_final = 100 + risk_free_rate

# Create a series for "Risk_Free" with increasing values
risk_free_values = np.linspace(risk_free_initial, risk_free_final, len(combined_historical2018_data))

# Add the "Risk_Free" series to the DataFrame
combined_historical2018_data['Risk_Free'] = risk_free_values

# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE  DBK.DE     VOW3.DE      BMW.DE      ADS.DE  \
2023-01-02  203.050003  10.942  120.040001   85.800003  127.699997   
2023-01-03  205.199997  11.112  122.059998   85.830002  131.880005   
2023-01-04  211.500000  11.698  125.879997   87.879997  138.380005   
2023-01-05  209.550003  11.494  127.120003   88.949997  138.539993   
2023-01-06  211.800003  11.596  128.160004   89.529999  140.679993   
...                ...     ...         ...         ...         ...   
2024-03-22  271.850006  14.178  118.080002  104.120003  200.250000   
2024-03-25  275.200012  14.146  119.379997  106.239998  203.949997   
2024-03-26  277.000000  14.230  121.260002  106.559998  204.300003   
2024-03-27  277.450012  14.612  121.760002  106.160004  204.699997   
2024-03-28  277.799988  14.582  122.839996  106.959999  207.000000   

                BEI.DE     DTE.SG      SAP.DE    1COV.DE     BAS.DE  EOAN.DE  \
2023-01-02  107.150002  18.910000   97.419998  38.259998  47.985001    9.468   

# Mean variance optimization

In [17]:
def mean_variance_optimization(expected_returns, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # start weighs
    bounds = [(0, 1)] * n  # weigh border (0-100% for each share)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x


# Portfilio allocation

In [18]:
def allocate_portfolio(predictions, historical_data, initial_capital):
    # Calculate expected returns based on the predictions
    expected_returns = predictions.mean()
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
           
    print (covariance_matrix)
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation


# Calculate portfolio

In [19]:
initial_capital = 1000 + profit # 1000€ start capital
print(initial_capital)
# Calculate the portfolio allocation
portfolio_allocation = allocate_portfolio(predictions_df, combined_historical2018_data, initial_capital)
# print portfolio allocation
print(portfolio_allocation)

1119.8380076835954
               ALV.DE     DBK.DE    VOW3.DE     BMW.DE      ADS.DE  \
ALV.DE     263.233687  16.722550 -36.220076   5.952651  172.491887   
DBK.DE      16.722550   1.551082   0.318550  -0.659780    6.540290   
VOW3.DE    -36.220076   0.318550  69.134268  19.760646  -63.554093   
BMW.DE       5.952651  -0.659780  19.760646  34.572252   35.942541   
ADS.DE     172.491887   6.540290 -63.554093  35.942541  272.479734   
BEI.DE     107.009092   4.771363 -30.880760  10.627189   99.324278   
DTE.SG      10.880760   0.681397  -0.801476  -0.534726    3.412920   
SAP.DE     279.440764  14.954034 -53.869391  32.660143  250.691795   
1COV.DE     46.889052   2.295622 -25.165169   0.619697   59.397072   
BAS.DE       4.412254   1.647993  17.191130   1.362113  -12.713713   
EOAN.DE      7.639689   0.216266  -2.472728   1.698286    9.464359   
RWE.DE     -32.281946  -1.548413   9.245094  -1.032059  -21.514112   
Risk_Free    1.251003   0.056769  -0.504171   0.075096    1.307821   



# Simulation for one month

In [20]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

0.0
0.0
0.0
-----------------------------
7.497600762796423e-08
1.0933001266423498e-06
1.0933001266423498e-06
-----------------------------
7.56989964845851e-09
9.29886445094843e-07
2.0231865717371927e-06
-----------------------------
1.0174708579761232e-09
1.0882868203760375e-07
2.1320152537747965e-06
-----------------------------
0.0
0.0
2.1320152537747965e-06
-----------------------------
0.0
0.0
2.1320152537747965e-06
-----------------------------
3.536006905536143e-08
7.933031303727195e-07
2.9253183841475162e-06
-----------------------------
0.0
0.0
2.9253183841475162e-06
-----------------------------
1.5169187629369437e-08
7.687744336857118e-07
3.694092817833228e-06
-----------------------------
0.5881595518622859
31.13128525956284
31.13128895365566
-----------------------------
79.62164501155934
1025.9249141978903
1057.056203151546
-----------------------------
1.9956071102031996
62.78179785995978
1119.8380010115059
-----------------------------
1.5986721774134995e-07
1.59867217

In [21]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

266.29998779296875
0.0
0.0
----------------------------
15.104000091552734
7.497600762796423e-08
1.1324376260770303e-06
----------------------------
120.75
7.56989964845851e-09
2.0465030086283957e-06
----------------------------
106.8000030517578
1.0174708579761232e-09
2.1551688993653203e-06
----------------------------
232.3000030517578
0.0
2.1551688993653203e-06
----------------------------
139.4499969482422
0.0
2.1551688993653203e-06
----------------------------
21.71999931335449
3.536006905536143e-08
2.923189574967938e-06
----------------------------
171.4199981689453
0.0
2.923189574967938e-06
----------------------------
47.54999923706055
1.5169187629369437e-08
3.6444844351712827e-06
----------------------------
49.05500030517578
0.5881595518622859
28.85217064058092
----------------------------
12.46500015258789
79.62164501155934
1021.3359878589669
----------------------------
33.0
1.9956071102031996
1087.1910224956725
----------------------------
100.33333333333333
1.598672177413

In [22]:
# just the profit, to store it
profit = new_portfolio_value - 1000

# path to JSON file
json_file_path = 'settings/profit.json'

# Create Dictionary with the value of new_portfolio_value
data = {'profit': profit}

# Write JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)

print(f'Wert von profit wurde erfolgreich in {json_file_path} gespeichert.')

Wert von profit wurde erfolgreich in settings/profit.json gespeichert.
